### First, we import the libraries that will help us in our task.

Pandas and Geopandas for our dataframes 🐼

Carto for amazing maps 🗺️

ReGex for our regular expressions 🆎

math for....well, math ➕

In [1]:
import re
import pandas as pd
import geopandas as gpd
from geopy.distance import distance
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
from pymongo import MongoClient
import math

import cartoframes
import cartoframes.viz
import cartoframes.viz.widgets

from cartoframes.viz import Map, Layer, size_continuous_style,color_continuous_style

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 25)



%matplotlib inline

### Our client is picky, and what he demand from us is to do an analysis to advice him in which city and place he should enplace the headquarters of his company. For this task we should take into consideration the following variables:

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.

In [2]:

client = MongoClient("mongodb://localhost/crunch")

db = client.get_database()

In [3]:

companies = list(db["crunch"].find({} \
            ,{"name":1,"offices":1,"category_code":1,"total_money_raised":1}))
len(companies)

18801

We have more than 18k companies in crunch databse! Let´s start working on it!

In [4]:
companies_df = pd.DataFrame(companies)

In [5]:
companies_df = companies_df.explode('offices')

In [6]:
companies_df_expanded = companies_df[["offices"]].apply(lambda x: x["offices"], result_type="expand", axis=1)

In [7]:
companies_df_clean = pd.concat([companies_df,companies_df_expanded],axis=1)

In [8]:
companies_df_clean = companies_df_clean.drop(columns=["_id","offices"])
companies_df_clean.head(5)

,name,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Wetpaint,web,$39.8M,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,Wetpaint,web,$39.8M,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,AdventNet,enterprise,$0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Zoho,software,$0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,Digg,news,$45M,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523


-----------------------

### We have our dataframe more cleaned, now we can start to make a scoring of cities. For that  we are going to filter the dataframe to find how much companies of gaming design we have.

In [9]:
companies_df_clean["category_code"].value_counts()

web                 4054
software            3392
mobile              1302
games_video         1200
advertising         1183
other               1155
enterprise           946
consulting           807
ecommerce            777
network_hosting      699
public_relations     619
search               496
hardware             461
biotech              402
cleantech            359
semiconductor        192
security             190
analytics             94
finance               66
news                  64
social                63
legal                 51
music                 50
messaging             47
education             43
photo_video           31
travel                29
health                28
medical               27
manufacturing         22
transportation        16
sports                14
hospitality           14
automotive            11
fashion               10
real_estate           10
design                 6
nanotech               5
nonprofit              4
government             1


In [10]:
companies_df_games = companies_df_clean[companies_df_clean["category_code"] == "games_video"]

In [11]:
companies_df_games["city"].value_counts()

New York                  75
San Francisco             68
                          54
London                    36
Los Angeles               31
                          ..
Macclesfiled, Cheshire     1
Lisbon                     1
Gurgaon                    1
Malmo                      1
Roncade TV                 1
Name: city, Length: 363, dtype: int64

In [12]:
scoring_df = pd.DataFrame(companies_df_games["city"].value_counts())
scoring_df = scoring_df.rename(columns={"city":"Design(Gaming) Companies"})

In [13]:
scoring_df

,Design(Gaming) Companies
New York,75
San Francisco,68
,54
London,36
Los Angeles,31
...,...
"Macclesfiled, Cheshire",1
Lisbon,1
Gurgaon,1
Malmo,1


### Now we will add to our scoring_df the companies with more than 1M of money raised

In [14]:
from functions import money

companies_df_clean["money raised"] = companies_df_clean["total_money_raised"].apply(money)

In [15]:
#companies_df_clean["money raised"]

In [16]:
from functions import units
    
companies_df_clean["units money raised"] = companies_df_clean["total_money_raised"].apply(units)

In [17]:
companies_df_clean = companies_df_clean[(companies_df_clean["units money raised"] == "B") | (companies_df_clean["units money raised"] == "M")]

In [18]:
scoring_invest = pd.DataFrame(companies_df_clean.city.value_counts())

In [19]:
scoring_invest

,city
San Francisco,436
New York,337
London,189
,117
Seattle,110
...,...
Tsimshatsui East,1
Vevey,1
"Haidian District,",1
Switzerland,1


In [20]:
scoring_df = scoring_df.merge(scoring_invest,right_index=True,left_index=True)

In [21]:
scoring_df = scoring_df.rename(columns={"city":"High Invested Companies"})

In [22]:
scoring_df

,Design(Gaming) Companies,High Invested Companies
New York,75,337
San Francisco,68,436
,54,117
London,36,189
Los Angeles,31,59
...,...,...
CH-8050 ZÃ¼rich,1,1
Petach-Tikva,1,3
Milan,1,4
Somerville,1,1


### Starbucks time!

In [23]:
strbcks = pd.read_csv('input/directory.csv') 

In [24]:
strbcks.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [25]:
strbcks_cities = pd.DataFrame(strbcks.City.value_counts())

In [26]:
strbcks_cities

,City
上海市,542
Seoul,243
北京市,234
New York,232
London,216
...,...
Lancashire,1
Saint Quentin en Yvelines,1
Barnegat,1
Elburn,1


In [27]:
scoring_df = scoring_df.merge(strbcks_cities,right_index=True,left_index=True)

In [28]:
scoring_df = scoring_df.rename(columns={"City":"Stabucks"})

In [29]:
#scoring_df.head(10)

### We are ready to make our scoring, but we just had a call with de CEO of the company and now he tells us that the most importat thing are "the gaming design in the city", followed by the "high invested companies" and at last place "starbucks are important, but not much"

In [30]:
weight_design_companies = 1
weight_invested_companies = 0.8
weight_starbucks_companies = 0.3


scoring_df["Scoring"] = (scoring_df["Design(Gaming) Companies"].rank(method='max',axis=0)*weight_design_companies) + \
                        (scoring_df["High Invested Companies"].rank(method='max',axis=0)*weight_invested_companies) + \
                        (scoring_df["Stabucks"].rank(method='max',axis=0)*weight_starbucks_companies)

In [31]:
scoring_df = scoring_df.sort_values("Scoring",ascending=False)

In [32]:
from functions import change_colour

scoring_df_style = scoring_df.style.apply(change_colour, axis=1, subset=['Scoring'])

In [33]:
scoring_df_style

,Design(Gaming) Companies,High Invested Companies,Stabucks,Scoring
New York,75,337,232,414.700000
London,36,189,216,411.600000
San Francisco,68,436,89,408.800000
Seattle,13,110,156,404.600000
Los Angeles,31,59,127,396.900000
Paris,17,63,61,392.500000
Chicago,8,57,180,390.800000
Austin,8,88,59,389.700000
San Jose,6,78,91,384.900000
Cambridge,8,77,23,380.300000


### Welcome to the home of the Empire State and Knicks! 🗽

-------------------------

### Now we will prepare de dataframe to be a shiny geodataframe

In [34]:
companies_df_clean = companies_df_clean.drop(columns="total_money_raised")

In [35]:
companies_df_clean.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,money raised,units money raised
0,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,39.80,M
0,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,39.80,M
3,Digg,news,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,45.00,M
4,Facebook,social,Headquarters,1601 Willow Road,,94025,Menlo Park,CA,USA,37.416050,-122.151801,2.43,B
4,Facebook,social,Europe HQ,,,,Dublin,None,IRL,53.344104,-6.267494,2.43,B


In [36]:
from functions import geo_Json

companies_df_clean["location"] = companies_df_clean[["latitude", "longitude"]].apply(lambda x: geo_Json(x.latitude,x.longitude), axis=1)


In [37]:
companies_df_clean = companies_df_clean[companies_df_clean["location"].notnull()]

Our first geodataframe!

In [38]:
gdf = gpd.GeoDataFrame(companies_df_clean, \
                       geometry=gpd.points_from_xy(companies_df_clean.longitude,companies_df_clean.latitude))

In [39]:
gdf = gdf[["name","category_code","city","money raised","geometry"]]

First we will get one geodataframe for every category

In [40]:
gdf_invested = gdf[(gdf["city"] == "New York")]

In [41]:
gdf_games = gdf_invested[(gdf_invested["category_code"] == "games_video")]

In [42]:
strbcks_filter = strbcks[strbcks["City"]=="New York"]

In [43]:
gdf_str = gpd.GeoDataFrame(strbcks_filter, geometry=gpd.points_from_xy(strbcks_filter.Longitude,strbcks_filter.Latitude))

In [44]:
analysis = Map([
    Layer(gdf_str,
        'color:green',
        default_widget=True),
    
    Layer(gdf_invested,
          'color:blue',
          default_widget=True),
    
    Layer(gdf_games,
          'color:black',
          default_widget=True)
    
],show_info=True
,default_legend=True
,title = "New York Data Analysis of startups with more than 1M money raised and with nearly Starbucks stores" )


### Now we can see the whole area and choose the better place for our client!

In [45]:
analysis

In [46]:
coords = {'lat': [40.739573], 'lng': [-73.990441]}
df_company = pd.DataFrame(data=coords)

In [47]:
gdf_company = gpd.GeoDataFrame(df_company, geometry=gpd.points_from_xy(df_company.lng,df_company.lat))

In [48]:
location_company_client = Map([
    Layer(gdf_str,
        'color:green',
        default_widget=True),
    
    Layer(gdf_invested,
          'color:blue',
          default_widget=True),
    
    Layer(gdf_games,
          'color:black',
          default_widget=True),
    
    Layer(gdf_company,
        'color:yellow',
        default_widget=True)
],show_info=True
,default_legend=True
,title = "New York Data Analysis of startups with more than 1M money raised and with nearly Starbucks stores" )


In [49]:
location_company_client